In [0]:
%sql
Create database if not exists Sample

In [0]:
%sql

CREATE TABLE IF NOT EXISTS Sample.Transactions ( AccountId INT, TranDate DATE, TranAmt DECIMAL(8, 2));

CREATE TABLE IF NOT EXISTS Sample.Logical (RowID INT,FName VARCHAR(20), Salary SMALLINT);

In [0]:
%sql

INSERT INTO Sample.Transactions VALUES 
( 1, '2011-01-01', 500),
( 1, '2011-01-15', 50),
( 1, '2011-01-22', 250),
( 1, '2011-01-24', 75),
( 1, '2011-01-26', 125),
( 1, '2011-01-28', 175),
( 2, '2011-01-01', 500),
( 2, '2011-01-15', 50),
( 2, '2011-01-22', 25),
( 2, '2011-01-23', 125),
( 2, '2011-01-26', 200),
( 2, '2011-01-29', 250),
( 3, '2011-01-01', 500),
( 3, '2011-01-15', 50 ),
( 3, '2011-01-22', 5000),
( 3, '2011-01-25', 550),
( 3, '2011-01-27', 95 ),
( 3, '2011-01-30', 2500)



num_affected_rows,num_inserted_rows
18,18


In [0]:
%sql
INSERT INTO Sample.Logical
VALUES (1,'George', 800),
(2,'Sam', 950),
(3,'Diane', 1100),
(4,'Nicholas', 1250),
(5,'Samuel', 1250),
(6,'Patricia', 1300),
(7,'Brian', 1500),
(8,'Thomas', 1600),
(9,'Fran', 2450),
(10,'Debbie', 2850),
(11,'Mark', 2975),
(12,'James', 3000),
(13,'Cynthia', 3000),
(14,'Christopher', 5000);

num_affected_rows,num_inserted_rows
14,14


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
df = spark.table("Sample.Transactions")
display(df)

AccountId,TranDate,TranAmt
1,2011-01-01,500.00
1,2011-01-15,50.00
1,2011-01-22,250.00
1,2011-01-24,75.00
1,2011-01-26,125.00
1,2011-01-28,175.00
2,2011-01-01,500.00
2,2011-01-15,50.00
2,2011-01-22,25.00
2,2011-01-23,125.00


In [0]:
window_spec = Window.partitionBy("AccountId").orderBy("TranDate")

In [0]:
df_window = df.withColumn(
    "Avg", avg("TranAmt").over(window_spec)
).withColumn(
    "Count", count("*").over(window_spec)
).withColumn(
    "Min", min("TranAmt").over(window_spec)
).withColumn(
    "Max", max("TranAmt").over(window_spec)
).withColumn(
    "Sum", sum("TranAmt").over(window_spec)
).withColumn(
    "Lead", lead("TranAmt").over(window_spec)
).withColumn(
    "Lag", lag("TranAmt").over(window_spec)
).withColumn(
    "First Value", first_value("TranAmt").over(window_spec)
).withColumn(
    "Last Value", last_value("TranAmt").over(window_spec)
).withColumn(
    "Row Number", row_number().over(window_spec)
)

df_window = df_window.orderBy("AccountId", "TranDate")

display(df_window)

AccountId,TranDate,TranAmt,Avg,Count,Min,Max,Sum,Lead,Lag,First Value,Last Value,Row Number
1,2011-01-01,500.00,500.000000,1,500.00,500.00,500.00,50.00,null,500.00,500.00,1
1,2011-01-15,50.00,275.000000,2,50.00,500.00,550.00,250.00,500.00,500.00,50.00,2
1,2011-01-22,250.00,266.666667,3,50.00,500.00,800.00,75.00,50.00,500.00,250.00,3
1,2011-01-24,75.00,218.750000,4,50.00,500.00,875.00,125.00,250.00,500.00,75.00,4
1,2011-01-26,125.00,200.000000,5,50.00,500.00,1000.00,175.00,75.00,500.00,125.00,5
1,2011-01-28,175.00,195.833333,6,50.00,500.00,1175.00,null,125.00,500.00,175.00,6
2,2011-01-01,500.00,500.000000,1,500.00,500.00,500.00,50.00,null,500.00,500.00,1
2,2011-01-15,50.00,275.000000,2,50.00,500.00,550.00,25.00,500.00,500.00,50.00,2
2,2011-01-22,25.00,191.666667,3,25.00,500.00,575.00,125.00,50.00,500.00,25.00,3
2,2011-01-23,125.00,175.000000,4,25.00,500.00,700.00,200.00,25.00,500.00,125.00,4
